## Practica 1 - Computer Vision

# Importación de librerías

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

# Exploración de archivos

In [2]:
# Rutas a los archivos de anotaciones
path_train = os.path.join(os.getcwd(), 'data', 'practica_1_dataset', 'train', 'annotations.csv')
path_valid = os.path.join(os.getcwd(), 'data', 'practica_1_dataset', 'valid', 'annotations.csv')
path_test = os.path.join(os.getcwd(), 'data', 'practica_1_dataset', 'test', 'annotations.csv')

# Cargar los DataFrames
df_train = pd.read_csv(path_train)
df_valid = pd.read_csv(path_valid)
df_test = pd.read_csv(path_test)

# Contar instancias por clase en cada DataFrame
train_class_counts = df_train['class'].value_counts()
valid_class_counts = df_valid['class'].value_counts()
test_class_counts = df_test['class'].value_counts()

# Combinar los DataFrames para contar las instancias generales por clase
df_combined = pd.concat([df_train, df_valid, df_test])
general_class_counts = df_combined['class'].value_counts()

# Obtener los valores únicos de la columna 'class' del conjunto combinado
valores_unicos = df_combined['class'].unique().tolist()

# Mostrar los resultados
print("Train class counts:")
print(train_class_counts)
print("\nValid class counts:")
print(valid_class_counts)
print("\nTest class counts:")
print(test_class_counts)
print("\nGeneral class counts:")
print(general_class_counts)
print("\nValores únicos en el dataset combinado:")
print(valores_unicos)

Train class counts:
class
fish         1961
jellyfish     385
penguin       330
shark         259
puffin        175
stingray      136
starfish       78
Name: count, dtype: int64

Valid class counts:
class
fish         459
jellyfish    155
penguin      104
puffin        74
shark         57
stingray      33
starfish      27
Name: count, dtype: int64

Test class counts:
class
fish         249
jellyfish    154
penguin       82
shark         38
puffin        35
stingray      15
starfish      11
Name: count, dtype: int64

General class counts:
class
fish         2669
jellyfish     694
penguin       516
shark         354
puffin        284
stingray      184
starfish      116
Name: count, dtype: int64

Valores únicos en el dataset combinado:
['starfish', 'shark', 'fish', 'puffin', 'stingray', 'penguin', 'jellyfish']


In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import shutil

# Rutas de los datos originales
base_dir = "data/practica_1_dataset"
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")

# Rutas de los datos procesados (donde se guardarán las imágenes originales y sintéticas)
processed_base = "data/processed"
processed_train_dir = os.path.join(processed_base, "train")
processed_validation_dir = os.path.join(processed_base, "validation")

# Se crean los directorios de salida si no existen
os.makedirs(processed_train_dir, exist_ok=True)
os.makedirs(processed_validation_dir, exist_ok=True)

# Mapeo de clases a números (para la máscara)
class_mapping = {
    "fish": 1,
    "jellyfish": 2,
    "penguin": 3,
    "shark": 4,
    "puffin": 5,
    "stingray": 6,
    "starfish": 7
}

def process_folder(input_folder, output_folder, annotations_df):
    """
    Procesa las imágenes de input_folder y guarda en output_folder:
      - Una copia de la imagen original.
      - Una imagen sintética en la que se "borran" (reemplazan) los píxeles de la clase predominante 
        usando el color medio calculado fuera de dicha región. Esta imagen se guarda con el sufijo 
        "_synthetic" antes de la extensión.
    Además, registra en un CSV la información (nombre de imagen, clase predominante y área) de cada imagen.
    """
    dataset_info = []  # Lista para almacenar la información de cada imagen procesada

    # Obtener archivos de imagen (considerando las extensiones jpg, jpeg y png)
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    for image_name in tqdm(image_files, desc=f"Procesando {os.path.basename(input_folder)}"):
        image_path = os.path.join(input_folder, image_name)
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Error al abrir {image_name}: {e}")
            continue

        width, height = image.size

        # Crear la máscara con fondo = 0
        mask = np.zeros((height, width), dtype=np.uint8)

        # Filtrar las anotaciones para la imagen
        image_annotations = annotations_df[annotations_df['filename'] == image_name]
        for _, row in image_annotations.iterrows():
            cls = row['class']
            if cls in class_mapping:
                class_id = class_mapping[cls]
                xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                mask[ymin:ymax, xmin:xmax] = class_id

        # Determinar la clase predominante (excluyendo el fondo)
        uniques, counts = np.unique(mask, return_counts=True)
        pixel_count_dict = dict(zip(uniques, counts))
        pixel_count_dict.pop(0, None)  # Se elimina el fondo
        if pixel_count_dict:
            predominant_class_id = max(pixel_count_dict, key=pixel_count_dict.get)
            predominant_area = pixel_count_dict[predominant_class_id]
            predominant_class_label = [label for label, idx in class_mapping.items() if idx == predominant_class_id][0]
        else:
            predominant_class_label = None
            predominant_area = 0

        # Registrar la información en el dataset
        dataset_info.append({
            "filename": image_name,
            "predominant_class": predominant_class_label,
            "area": predominant_area
        })

        # Copiar la imagen original al directorio procesado
        dest_original_path = os.path.join(output_folder, image_name)
        shutil.copy(image_path, dest_original_path)

        # Generar la imagen sintética
        image_array = np.array(image)
        if predominant_class_label is not None:
            mask_predominant = (mask == predominant_class_id)
            # Calcular el color medio de los píxeles fuera de la región predominante
            if np.any(~mask_predominant):
                mean_color = image_array[~mask_predominant].mean(axis=0).astype(np.uint8)
            else:
                mean_color = np.array([0, 0, 0], dtype=np.uint8)
            synthetic_image_array = image_array.copy()
            synthetic_image_array[mask_predominant] = mean_color
        else:
            synthetic_image_array = image_array.copy()

        # Se genera el nombre de la imagen sintética con el sufijo "_synthetic" antes de la extensión .jpg
        file_base, _ = os.path.splitext(image_name)
        synthetic_name = f"{file_base}_synthetic.jpg"
        synthetic_image = Image.fromarray(synthetic_image_array)
        synthetic_image.save(os.path.join(output_folder, synthetic_name))
    
    # Guardar el CSV con la información de las imágenes en el mismo directorio de salida
    csv_output_path = os.path.join(output_folder, "new_dataset.csv")
    pd.DataFrame(dataset_info).to_csv(csv_output_path, index=False)
    print(f"Procesamiento completado para {os.path.basename(input_folder)}. CSV guardado en {csv_output_path}")

# Leer los archivos de anotaciones respectivos de cada carpeta
train_annotations_path = os.path.join(train_dir, "annotations.csv")
valid_annotations_path = os.path.join(valid_dir, "annotations.csv")

train_annotations = pd.read_csv(train_annotations_path)
valid_annotations = pd.read_csv(valid_annotations_path)

# Procesar el conjunto de entrenamiento
process_folder(train_dir, processed_train_dir, train_annotations)

# Procesar el conjunto de validación (se guarda en la carpeta "validation")
process_folder(valid_dir, processed_validation_dir, valid_annotations)

print("Todos los conjuntos han sido procesados correctamente.")

Procesando train: 100%|██████████| 448/448 [00:21<00:00, 20.59it/s]


Procesamiento completado para train. CSV guardado en data/processed\train\new_dataset.csv


Procesando valid: 100%|██████████| 127/127 [00:05<00:00, 23.30it/s]

Procesamiento completado para valid. CSV guardado en data/processed\validation\new_dataset.csv
Todos los conjuntos han sido procesados correctamente.
